<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/F1_LRAP_for_Value_Det1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!apt-get -qq install -y unrar


In [3]:
import os


In [4]:
!pip install rarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import rarfile

# rar_file_path = '/content/drive/MyDrive/transformers/model-rob.rar'
# with rarfile.RarFile(rar_file_path) as rf:
#     rf.extractall('/content/drive/MyDrive')

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [7]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', '/content/drive/MyDrive/mmm/u', num_labels=21, args={'train_batch_size':16, 'overwrite_output_dir': True, 'num_train_epochs': 5})



In [ ]:
!pip list

In [10]:
!pip show pandas

Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.8/dist-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, arviz, cmdstanpy, cufflinks, datascience, datasets, db-dtypes, evaluate, fastai, fix-yahoo-finance, google-colab, gspread-dataframe, holoviews, mizani, mlxtend, pandas-datareader, pandas-gbq, pandas-profiling, plotnine, prophet, pymc, seaborn, simpletransformers, sklearn-pandas, statsmodels, streamlit, vega-datasets, xarray


In [11]:
!pip show simpletransformers

Name: simpletransformers
Version: 0.63.9
Summary: An easy-to-use wrapper library for the Transformers library.
Home-page: https://github.com/ThilinaRajapakse/simpletransformers/
Author: Thilina Rajapakse
Author-email: chaturangarajapakshe@gmail.com
License: 
Location: /usr/local/lib/python3.8/dist-packages
Requires: datasets, numpy, pandas, regex, requests, scikit-learn, scipy, sentencepiece, seqeval, streamlit, tensorboard, tokenizers, tqdm, transformers, wandb
Required-by: 


In [12]:
!pip show transformers

Name: transformers
Version: 4.26.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /usr/local/lib/python3.8/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: simpletransformers


In [14]:
!pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: kapre


In [16]:
import pandas as pd

In [21]:
import numpy as np

In [29]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-training.tsv', sep='\t') 
arguments_validation_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/drive/MyDrive/transformers/arguments-test.tsv')
arguments_test_data_nahjalbalagha= pd.read_table('/content/drive/MyDrive/transformers/arguments-test-nahjalbalagha.tsv')

labels_training = pd.read_csv('/content/drive/MyDrive/transformers/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/drive/MyDrive/transformers/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)

print(labels_training.shape)
labels_training.head()


#new added label

label_to_add = ["Stance"]

# fixing the dataset for inconsistency: in favor of, in favour of
arguments_training_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)


arguments_test_data['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

# from categorical value to numerical value for Stance

arguments_training_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)


arguments_test_data['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favor of','against'],
                                                    [1,0], inplace=True)



(5393, 20)


In [ ]:
labels_training['Stance_As_Label'] = arguments_training_no_label[label_to_add] 
print(labels_training.shape)
print(labels_training.head())



labels_validation['Stance_As_Label'] = arguments_validation_no_label[label_to_add]
print(labels_validation.shape)
print(labels_validation.head())


In [ ]:
# training set
train_labels = labels_training.values.tolist()

cols_to_concat = ["Conclusion", "Premise"]

arguments_training_no_label['Concated'] = arguments_training_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

print(arguments_training_no_label['Concated'][0])

#lowercase every word - roberta is case sensitive, thats why
arguments_training_no_label['Concated']= arguments_training_no_label['Concated'].str.lower()
print(arguments_training_no_label['Concated'][0])

train_text = arguments_training_no_label['Concated'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)] 
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]


In [32]:
# repeat above for the validation set
val_labels = labels_validation.values.tolist()

arguments_validation_no_label['Concated'] = arguments_validation_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)
print(arguments_validation_no_label['Concated'][0])

arguments_validation_no_label['Concated']= arguments_validation_no_label['Concated'].str.lower()
print(arguments_validation_no_label['Concated'][0])


val_text = arguments_validation_no_label['Concated'].values
val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]




Entrapment should be legalized</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
entrapment should be legalized</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?


In [33]:
# repeat above for the main test set

arguments_test_data['Concated'] = arguments_test_data[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

test_data = arguments_test_data['Concated'].values
print(np.shape(test_data))

print(arguments_test_data['Concated'][0])
arguments_test_data['Concated']= arguments_test_data['Concated'].str.lower()
print(arguments_test_data['Concated'][0])

(1576,)
We should end affirmative action</s>affirmative action helps with employment equity.
we should end affirmative action</s>affirmative action helps with employment equity.


In [34]:
# repeat above for the nahjalbalagha test set

arguments_test_data_nahjalbalagha['Concated'] = arguments_test_data_nahjalbalagha[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

test_data_nahjalbalagha = arguments_test_data_nahjalbalagha['Concated'].values
print(np.shape(test_data_nahjalbalagha))
print(test_data_nahjalbalagha[0])

arguments_test_data_nahjalbalagha['Concated']= arguments_test_data_nahjalbalagha['Concated'].str.lower()
print(arguments_test_data_nahjalbalagha['Concated'][0])

(279,)
Habitual greed devalues humans</s>Greed is like a slippery rock on which even the steps of scholars are not firm.
habitual greed devalues humans</s>greed is like a slippery rock on which even the steps of scholars are not firm.


In [48]:
# for LRAP, with Stance predicted as well
result, model_outputs, wrong_predictions = model.eval_model(val_df)

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [49]:
result

{'LRAP': 0.7943524162106567, 'eval_loss': 0.3051606677001036}

In [24]:
from sklearn.metrics import f1_score

In [42]:
# predicted labels and true labels for the validation set for F1 score, without stance

# f1_score functions takes np.array

#text = val_df['text'].to_list()

preds, _ = model.predict(list(val_df['text']))
preds= np.array(preds)
preds_without_last_col = preds[:, :-1]

true_labels =np.array(val_labels)
true_labels_without_last_col=true_labels[:,:-1]


# F1 score for each label separately
f1_scores = f1_score(true_labels_without_last_col, preds_without_last_col, average=None)

# average F1 score across all labels
average_f1_score = np.mean(f1_scores)


  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
print(preds_without_last_col.shape)
print(true_labels_without_last_col.shape)

In [45]:
average_f1_score

0.45265694175648613

In [ ]:
f1_scores

In [50]:
# with Stance prediction f1 score

preds, _ = model.predict(list(val_df['text']))
preds= np.array(preds)
true_labels =np.array(val_labels)

# F1 score for each label separately
f1_scores = f1_score(true_labels, preds, average=None)

# average F1 score across all labels
average_f1_score = np.mean(f1_scores)

  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [51]:
average_f1_score

0.47615168317893475